In [1]:
import pandas as pd
import os
import numpy as np
import os

# Get the current working directory
current_directory = os.getcwd()
print(current_directory)
# Parameter windowing
input_width = 48
label_width = 48
shift = label_width
total_window_size = input_width + shift
OUT_STEPS = label_width
# Definisikan irisan untuk input dan label
input_slice = slice(0, input_width)
label_start = total_window_size - label_width
labels_slice = slice(label_start, None)

def import_data(pathload, hari_khusus_saja=False):
    df_awal=pd.read_excel(pathload)
    #DROP KOLOM SUHU
    df_awal.drop(columns=['Keterangan', 'Keterangan.1'], inplace=True)
    df_awal.dropna(subset=['Beban'], inplace=True)
    if hari_khusus_saja==True:
        return df_awal[df_awal['Hari_Khus']!=0].copy()
    else:
        return df_awal.copy()
    
    
def datetime_kolom(data, jumlah_hari, kelipatan_hari):
    df=data.copy()
    df['Date']=pd.to_datetime(df['Date'], format="%Y-%m-%d %H:%M:%S", dayfirst=True) #year-month-day(tanggal bukan hari)  H= hours, M=minutes, S= sekon 
    df['Tanggal']=df['Date'].dt.day
    df['Bulan']=df['Date'].dt.month
    df['Tahun']=df['Date'].dt.year
    df['Jam']=df['Date'].dt.hour
    df['Menit']=df['Date'].dt.minute
    
    df['Jam']=df['Jam']+df['Menit']/60 # agar setiap 30 menit menjadi 0.5 jam (contoh jam 13:30 menjadi 13.5)
    df.drop(columns=['Menit'], inplace=True)

    #1 hari = 48 (karena 30 menit), 1 bulan = 30 hari, 1 tahun =12 bulan
    mod_data= df['Beban'].count() % 48
    smpl_dta=df['Date'].count()-df['Beban'].count()+mod_data+(48*jumlah_hari*kelipatan_hari)
    df=df[-smpl_dta:]
    return df.copy()


def make_windows(data_x,data_y, total_window_size, input_slice, labels_slice):
    x = []
    y = []
    for i in range(len(data_x) - total_window_size + 1):
        window_x = data_x[i:i+total_window_size]
        x.append(window_x[input_slice])

    for i in range(len(data_y) - total_window_size + 1):
        window_y= data_y[i:i+total_window_size]
        y.append(window_y[labels_slice])

    return np.array(x), np.array(y)
def sin_cos_time(data):
    df= data.copy()
    df['Jam_sin'] = np.sin(df['Jam'] * (2 * np.pi / 24))
    df['Jam_cos'] = np.cos(df['Jam'] * (2 * np.pi / 24))
    df['Hari_sin'] = np.sin((df['Hari']-1) * (2 * np.pi / 7))
    df['Hari_cos'] = np.cos((df['Hari']-1) * (2 * np.pi / 7))
    df['Hari_Khus_sin'] = np.sin((df['Hari_Khus']) * (2 * np.pi / 23))
    df['Hari_Khus_cos']= np.cos((df['Hari_Khus']) * (2 * np.pi / 23))
    df['Tanggal_sin'] = np.sin((df['Tanggal']-1) * (2 * np.pi / 30))
    df['Tanggal_cos'] = np.cos((df['Tanggal']-1) * (2 * np.pi / 30))
    df['Bulan_sin'] = np.sin((df['Bulan']-1) * (2 * np.pi / 12))
    df['Bulan_cos'] = np.cos((df['Bulan']-1) * (2 * np.pi / 12))
    return df.copy()
    

d:\KULIAH S2\Komputasi Lanjut\CODE\ANN


In [2]:
df_awal                         =   import_data(pathload='D:\\KULIAH S2\\Komputasi Lanjut\\CODE\\cuaca_bebanLR5_yangbaru.xlsx', hari_khusus_saja=False)
df_awal

,Date,JKT_suhu,JKT_hum,JKT_rain,JBR_suhu,JBR_hum,JBR_rain,JTG_suhu,JTG_hum,JTG_rain,JTM_suhu,JTM_hum,JTM_rain,BLI_suhu,BLI_hum,BLI_rain,Hari,Hari_Khus,Beban
0,2022-01-01 00:30:00,25.25,85.5,0.00,21.25,97.5,0.0,23.00,98.0,0.0,24.75,93.0,0.0,25.70,87.0,0.0,6,2,18402.00
1,2022-01-01 01:00:00,25.10,86.0,0.00,21.10,98.0,0.0,22.90,98.0,0.0,24.80,92.0,0.0,25.80,86.0,0.0,6,2,18214.00
2,2022-01-01 01:30:00,25.15,86.0,0.00,21.05,97.0,0.0,23.15,96.5,0.0,24.80,93.0,0.1,25.80,86.5,0.0,6,2,17974.00
3,2022-01-01 02:00:00,25.20,86.0,0.00,21.00,96.0,0.0,23.40,95.0,0.0,24.80,94.0,0.2,25.80,87.0,0.0,6,2,17729.00
4,2022-01-01 02:30:00,25.15,86.0,0.00,20.90,95.5,0.0,23.35,95.5,0.0,24.65,94.5,0.1,25.85,86.5,0.0,6,2,17532.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37079,2024-02-12 12:00:00,30.20,67.0,0.10,26.90,68.0,0.0,29.30,78.0,0.2,31.10,63.0,0.0,29.30,74.0,0.0,1,0,26775.07
37080,2024-02-12 12:30:00,30.60,65.0,0.10,27.50,65.5,0.0,29.45,77.0,0.2,31.60,60.5,0.0,29.50,73.0,0.0,1,0,26803.02
37081,2024-02-12 13:00:00,31.00,63.0,0.10,28.10,63.0,0.0,29.60,76.0,0.2,32.10,58.0,0.0,29.70,72.0,0.0,1,0,27538.69
37082,2024-02-12 13:30:00,31.40,61.0,0.05,28.15,62.5,0.0,29.65,75.5,0.2,32.20,57.0,0.0,29.80,71.5,0.0,1,0,28408.38


In [3]:
df_awal['Hari_Khus'].unique()

array([ 2,  0,  3,  4,  5,  1,  6,  8,  7,  9, 18, 10, 11, 12, 13, 14, 15,
       16, 17, 20, 21, 22, 23], dtype=int64)

In [4]:
df_1=datetime_kolom(data=df_awal, jumlah_hari=365, kelipatan_hari=1)
df_1 =df_1[['Date','Bulan','Tanggal','Hari', 'Jam','Beban', "Hari_Khus"]].copy()
df_1

,Date,Bulan,Tanggal,Hari,Jam,Beban,Hari_Khus
19536,2023-02-12 00:30:00,2,12,7,0.5,20312.00,0
19537,2023-02-12 01:00:00,2,12,7,1.0,20180.00,0
19538,2023-02-12 01:30:00,2,12,7,1.5,20021.00,0
19539,2023-02-12 02:00:00,2,12,7,2.0,19906.00,0
19540,2023-02-12 02:30:00,2,12,7,2.5,19743.00,0
...,...,...,...,...,...,...,...
37079,2024-02-12 12:00:00,2,12,1,12.0,26775.07,0
37080,2024-02-12 12:30:00,2,12,1,12.5,26803.02,0
37081,2024-02-12 13:00:00,2,12,1,13.0,27538.69,0
37082,2024-02-12 13:30:00,2,12,1,13.5,28408.38,0


In [5]:
df_2 = sin_cos_time(data=df_1)
cutoff = pd.to_datetime('2024-01-01 00:00:00')
df_3= df_2[df_2['Date'] <= cutoff].copy()
df_3.drop(columns=['Bulan', 'Tanggal','Hari', 'Jam'], inplace=True)
df_3.set_index('Date', inplace=True)
df_3

,Beban,Hari_Khus,Jam_sin,Jam_cos,Hari_sin,Hari_cos,Hari_Khus_sin,Hari_Khus_cos,Tanggal_sin,Tanggal_cos,Bulan_sin,Bulan_cos
Date,,,,,,,,,,,,
2023-02-12 00:30:00,20312.00,0,0.130526,0.991445,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 01:00:00,20180.00,0,0.258819,0.965926,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 01:30:00,20021.00,0,0.382683,0.923880,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 02:00:00,19906.00,0,0.500000,0.866025,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
2023-02-12 02:30:00,19743.00,0,0.608761,0.793353,-0.781831,0.62349,0.0,1.0,7.431448e-01,-0.669131,0.5,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 22:00:00,21241.27,0,-0.500000,0.866025,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025
2023-12-31 22:30:00,20800.42,0,-0.382683,0.923880,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025
2023-12-31 23:00:00,20450.02,0,-0.258819,0.965926,-0.781831,0.62349,0.0,1.0,-2.449294e-16,1.000000,-0.5,0.866025


In [6]:
df_3.to_excel('cuaca_bebanLR5_v2.xlsx')